In [1]:
// 1. Installation

//install libraries
import org.archive.archivespark._
import org.archive.archivespark.functions._
import org.archive.archivespark.specific.warc._

// data files - generic path from docker run -v
val cdxPath = "/data/arc_cdx/*.cdx"
val warcPath = "/data/warc"

import org.apache.spark.sql.{Row, SparkSession}

val session = spark.newSession

// collect all records

val r = ArchiveSpark.load(WarcSpec.fromFiles(cdxPath, warcPath))

In [2]:
val researchYear = "1997"

In [3]:
// 2. Count ALL objects from research Year
// 2.1 get data for text/html & HTTP 200 for each Year
val t1 = r.filter(r => r.timestamp.startsWith(researchYear))


In [4]:
// 3. generate basic data frame

val m = t1.map(f=> (f.originalUrl,f.digest,f.status,f.mime))
val df = session.createDataFrame(m).toDF("originalUrl","digest","status","mime").cache()

In [5]:
import org.apache.spark.sql.functions.count
import org.apache.spark.sql.functions._


In [6]:
df.filter(df("mime") === "text/html").select(countDistinct("originalUrl")).show()

+---------------------------+
|count(DISTINCT originalUrl)|
+---------------------------+
|                     658493|
+---------------------------+



In [7]:
df.select(countDistinct("originalUrl")).show()

+---------------------------+
|count(DISTINCT originalUrl)|
+---------------------------+
|                    1087617|
+---------------------------+



In [8]:
df.groupBy("mime").agg(countDistinct("originalUrl") as "URLs").orderBy(desc("URLs")).show(100,false)

+---------------------------------+------+
|mime                             |URLs  |
+---------------------------------+------+
|text/html                        |658493|
|image/gif                        |243201|
|image/jpeg                       |116268|
|text/plain                       |27526 |
|application/zip                  |8735  |
|application/pdf                  |8453  |
|application/octet-stream         |4159  |
|application/postscript           |4061  |
|audio/x-midi                     |2977  |
|audio/x-wav                      |2106  |
|audio/x-mid                      |1787  |
|application/x-tar                |1749  |
|image/x-xbitmap                  |1084  |
|application/x-gzip               |963   |
|audio/midi                       |904   |
|application/x-dvi                |684   |
|image/x-xpixmap                  |494   |
|application/x-tex                |483   |
|audio/x-pn-realaudio             |393   |
|application/mac-binhex40         |366   |
|audio/x-mo

In [9]:
df.groupBy("mime").agg(countDistinct("originalUrl") as "URLs").count()

111